In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
path = os.path.join('data','train.csv')
data = pd.read_csv(path)
data.isna().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [5]:
data = data.dropna()

In [6]:
data.columns

Index(['textID', 'text', 'selected_text', 'sentiment'], dtype='object')

In [7]:
data = data.drop(columns=['textID','text'])

In [8]:
query = data['selected_text']

In [9]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
target = encoder.fit_transform(data[['sentiment']])
target = target.toarray()

In [10]:
query = np.array(query)

In [11]:
from nltk.tokenize import word_tokenize
import string

In [12]:
def removing_punctuation(text):
    tokens = word_tokenize(text.lower())
    tokens_without_punctuation = [word for word in tokens if word not in string.punctuation]
    return ' '.join(tokens_without_punctuation)

punchuation_query = [removing_punctuation(sentence) for sentence in query]

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
stopword_query = []
stop_words = set(stopwords.words('english'))

for item in punchuation_query: 
    tokenized_list = tokenizer.tokenize(item.lower())
    filtered_tokens = [token for token in tokenized_list if token not in stop_words]
    joined_string = ' '.join(filtered_tokens)
    stopword_query.append(joined_string)

NameError: name 'punchuation_query' is not defined

In [134]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(stopword_query)
sequences = tokenizer.texts_to_sequences(stopword_query)
text_data = pad_sequences(sequences, maxlen=max_len)
print(text_data[:10])

[[   0    0    0 ...    0    0   10]
 [   0    0    0 ...    0  307   25]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0  359  461]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...   15 2480 2909]]


In [135]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(text_data,target,train_size=0.8,random_state=42)

In [136]:
from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model.add(layers.Dense(3,activation='softmax'))

In [137]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [139]:
model.fit(X_train, y_train, epochs=1,validation_split=0.1)

619/619 [==============================] - 26s 38ms/step - loss: 0.4937 - accuracy: 0.8102 - val_loss: 0.4759 - val_accuracy: 0.8113


In [ ]:
prediction = model.predict(X_test)
prediction = encoder.inverse_transform(prediction)

172/172 [==============================] - 3s 20ms/step


In [ ]:
prediction[250]

array(['neutral'], dtype=object)

In [ ]:
print(encoder.inverse_transform(y_test)[250])

['neutral']


In [ ]:
model.save('sentiment.h5')

c:\Users\ssnow\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model

model = load_model('sentiment.h5')

In [140]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def removing_punctuation(text):
    tokens = word_tokenize(text.lower())
    tokens_without_punctuation = [word for word in tokens if word not in string.punctuation]
    return ' '.join(tokens_without_punctuation)

def remove_stopwords(text):
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    tokenized_list = tokenizer.tokenize(text.lower())
    filtered_tokens = [token for token in tokenized_list if token not in stop_words]
    return ' '.join(filtered_tokens)

model = load_model('sentiment.h5', compile=False)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
user_input = input('enter message')

def checkingFunction(user_input):    
    processed_input = removing_punctuation(user_input)
    processed_input = remove_stopwords(processed_input)

    max_words = 5000
    max_len = 200

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(stopword_query)  

    sequences = tokenizer.texts_to_sequences([processed_input])
    text_data = pad_sequences(sequences, maxlen=max_len)

    prediction = model.predict(text_data)

    encoder = OneHotEncoder()
    encoder.fit(data[['sentiment']])  

    predicted_label = encoder.inverse_transform(prediction)
    
    if predicted_label == 'negative':
        return True
    else:
        return False

check_result = (checkingFunction(user_input))

while True:
    if check_result == True:
        print('block')
        break
    else:
        print('safe message')
        user_input = input('enter message')
        check_result = (checkingFunction(user_input))



1/1 [==============================] - 1s 963ms/step
safe message
1/1 [==============================] - 0s 33ms/step
safe message
1/1 [==============================] - 0s 29ms/step
block
